In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
# ensure that any edits to libraries you make are reloaded here automatically,
# and also that any charts or images displayed are shown in this notebook.


import numpy as np
import scipy as sc
import sklearn as sk
from sklearn import datasets, linear_model
import pandas as pd
import datetime as dt

import matplotlib.pyplot as plt
import matplotlib #as mpl

np.set_printoptions(precision=2)
plt.rcParams['figure.figsize'] = [12, 8]


## Load data into pd dataframes ##

# nyt case and death counts, where each date-county pair is a row
nyt_counts_df = pd.read_csv('data/nyt-us-counties-cases-deaths.csv') # must remove non-ascii
nyt_counts_df['date'] = pd.to_datetime(nyt_counts_df['date'])

# contains ordinal dates (t=1 on jan 1, 0001)
interventions_df = pd.read_csv('data/jieyingwu_dates_interventions.csv')

# ex/ pop size, income, urban/rural, temperature, public transit score, doctors per 1000 people
demographics_df = pd.read_csv('data/jieyingwu_demographics_counties.csv') 


# make pivot df where each county is a row, with dates as columns
cases_df = pd.pivot_table(nyt_counts_df, values = 'cases', index=['fips','county','state'], columns = 'date').reset_index()
deaths_df = pd.pivot_table(nyt_counts_df, values = 'deaths', index=['fips','county','state'], columns = 'date').reset_index()


# merge in additional features (case/death counts will be rightmost columns)
# first 3 cols will be fips, county, state
cases_df = interventions_df.merge(cases_df, left_on='FIPS', right_on='fips', suffixes=('_interventions', '_cases')) 
cases_df = demographics_df.merge(cases_df, left_on='FIPS', right_on='FIPS', suffixes=('_demog', '_cases')) 

deaths_df = interventions_df.merge(deaths_df, left_on='FIPS', right_on='fips',suffixes=('_interventions', '_deaths')) 
deaths_df = demographics_df.merge(deaths_df, left_on='FIPS', right_on='FIPS', suffixes=('_demog', '_cases')) 

# which cols in cases_df are counts
#counts_mask = [str(c)[:5] == '2020-' for c in cases_df.columns]




####### rest of code uses cases_df only for now

# TODO maybe exclude counties that are missing counts for most dates before filtering on missingness

# extract numeric columns as features
# remove fips, which is technically numeric (TODO check for other such cols)
not_fips_mask = [str(c).lower() != 'fips' for c in cases_df.columns]
features_target_df = cases_df.loc[:,not_fips_mask]
features_target_df = features_target_df.select_dtypes(include='number') # numeric only

# keep columns which are >10% missing (lose a lot of prev dates)
keep_cols = features_target_df.columns[features_target_df.isnull().mean() < 0.1]
features_target_df = features_target_df[keep_cols]

print(features_target_df.columns)


Index(['Rural-urban_Continuum Code_2013',       'Urban_Influence_Code_2013',
                'Economic_typology_2015',               'POP_ESTIMATE_2018',
                        'N_POP_CHG_2018',                     'Births_2018',
                           'Deaths_2018',                'NATURAL_INC_2018',
                'INTERNATIONAL_MIG_2018',               'DOMESTIC_MIG_2018',
       ...
                    'foreign travel ban',               2020-04-05 00:00:00,
                     2020-04-06 00:00:00,               2020-04-07 00:00:00,
                     2020-04-08 00:00:00,               2020-04-09 00:00:00,
                     2020-04-10 00:00:00,               2020-04-11 00:00:00,
                     2020-04-12 00:00:00,               2020-04-13 00:00:00],
      dtype='object', length=261)


In [3]:
# try linear on prev counts in same county, see if later dates have stronger effect

end = pd.to_datetime('2020-04-12 00:00:00') # pd slicing includes last index
x_df = features_target_df.loc[:, :end]

counts_mask = [str(c)[:5] == '2020-' for c in x_df.columns]
x_df = x_df.iloc[:,counts_mask]
x = x_df.values # ? can directly run regression on df
y = features_target_df.iloc[:,-1].values # last day
y = y.reshape((-1,1))

# mask counties with missing values
# can also mask counties w missing y here
mask = ~np.isnan(x).any(axis=1) & ~np.isinf(x).any(axis=1) & ~np.isneginf(x).any(axis=1) 


regr = linear_model.LinearRegression() 
regr.fit(x[mask], y[mask])

#preds = regr.predict(test_data)


# print top coefficients
col_coefL = list( zip(list(x_df.columns), regr.coef_.flatten().tolist()) )
col_coefL.sort(key=lambda tup: abs(tup[1]), reverse=True)
for tup in col_coefL:
    print(tup)

(Timestamp('2020-04-12 00:00:00'), 1.1907482011739179)
(Timestamp('2020-04-05 00:00:00'), -0.3697169133785038)
(Timestamp('2020-04-06 00:00:00'), 0.3016810900062734)
(Timestamp('2020-04-09 00:00:00'), -0.29756291430070575)
(Timestamp('2020-04-08 00:00:00'), 0.29135366040273675)
(Timestamp('2020-04-10 00:00:00'), -0.16354517040375438)
(Timestamp('2020-04-11 00:00:00'), 0.1260647183145272)
(Timestamp('2020-04-07 00:00:00'), -0.10301216069660735)


In [8]:
# TODO split train test - currently train on all counties

## linear model: log(deaths) against demographics, interventions, and all deaths in all counties on previous days

end = pd.to_datetime('2020-04-12 00:00:00') # pd slicing includes last index
x_df = features_target_df.loc[:, :end]
x = x_df.values # ? can directly run regression on df
y = features_target_df.iloc[:,-1].values # last day
y = y.reshape((-1,1))

# mask counties with missing values
# can also mask counties w missing y here
mask = ~np.isnan(x).any(axis=1) & ~np.isinf(x).any(axis=1) & ~np.isneginf(x).any(axis=1) 


regr = linear_model.LinearRegression() 
regr.fit(x[mask], y[mask])

#preds = regr.predict(test_data)


print('top coefficients')
col_coefL = list( zip(list(x_df.columns), regr.coef_.flatten().tolist()) )
col_coefL.sort(key=lambda tup: abs(tup[1]), reverse=True)
for tup in col_coefL:
    print(tup)

# clearly overfitting 
# ex/ top 3 coefficients: 
#('Area in square miles - Water area', -51.06425376228152)
#('Area in square miles - Total area', 51.06322457784346)
#('Area in square miles - Land area', -51.06304497658695)


top coefficients
('Area in square miles - Water area', -51.06425376228152)
('Area in square miles - Total area', 51.06322457784346)
('Area in square miles - Land area', -51.06304497658695)
('Fraction of Active Physicians Who Are Age 60 or Older 2018 (AAMC)', 38.915850822604284)
('Fraction of Residents in ACGME Programs Who Are IMGs as of December 31 2018 (AAMC)', 34.122441652076134)
('Fraction of Active Physicians Who Are International Medical Graduates (IMGs) 2018 (AAMC)', -33.41748141277473)
('Fraction of Active Physicians Who Are Female 2018 (AAMC)', 20.428786823122678)
('Fraction of Physicians Retained in State from Undergraduate Medical Education (UME) 2018 (AAMC)', 9.733281160778185)
('Active Patient Care General Surgeons per 100000 Population 2018 (AAMC)', 7.077736748772218)
('Active General Surgeons per 100000 Population 2018 (AAMC)', -6.3752646327514615)
('Neurological Surgery (AAMC)', 3.952168651194428)
('R_INTERNATIONAL_MIG_2018', 3.899061026805068)
('Nephrology (AAMC)', -3.

In [9]:
# try lasso # TODO doesn't converge

regr = linear_model.Lasso()
regr.fit(x[mask], y[mask])



print('top (10) coefficients, lasso')
col_coefL = list( zip(list(x_df.columns), regr.coef_.flatten().tolist()) )
col_coefL.sort(key=lambda tup: abs(tup[1]), reverse=True)
for tup in col_coefL[:10]:
    print(tup)


top (10) coefficients, lasso
('General Surgery (AAMC)', -2.6491216384492353)
('Dermatology (AAMC)', -1.6502319217682293)
('Urology (AAMC)', -1.5359075508582944)
('Radiology & Diagnostic Radiology (AAMC)', -1.5022985736838645)
('Orthopedic Surgery (AAMC)', -1.4888701490555338)
('MURDER', -1.1859337698394574)
('Ophthalmology (AAMC)', -1.1384883354492206)
('Neuroradiology (AAMC)', 1.1266359224205704)
('Child & Adolescent Psychiatry** (AAMC)', 1.0837118839667557)
('Hematology & Oncology (AAMC)', 0.8226595102969815)


/home/lgai/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 529732.1544252624, tolerance: 278096.8752547469
  positive)


In [1]:
# convert dates to numbers for plt
x = matplotlib.dates.date2num(train_cases_df.columns[3:])
ys = train_cases_df.iloc[:,3:]
ys = ys[::20]

## linear model: regress log(deaths) against demographics + interventions + counts
## linear model: regress log(deaths) against deaths in all counties on previous days
#            maybe too few sample - coeffs would be different for pred LA vs other county
## rf, arima, curve fitting

## regression, arimax model. 
# log(cases_county_x) = beta * X_demographic + phi_(t-1) * cases_(all counties, t-1) + ....

NameError: name 'matplotlib' is not defined